In [1]:
import os
import pandas as pd
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms, models
from tqdm import tqdm  # for a nice progress bar


In [2]:
device = "mps" if torch.backends.mps.is_available() else ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: mps


In [3]:
base_dir = os.getcwd()
train_csv = os.path.join(base_dir, "train", "train_data.csv")
train_dir = os.path.join(base_dir, "train", "train")
test_csv = os.path.join(base_dir, "test", "test.csv")
test_dir = os.path.join(base_dir, "test", "test")


In [3]:
os.listdir(./desktop/train/train


In [16]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, mode='train'):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.mode = mode
        
        if self.mode == 'train':
            self.classes = sorted(self.data['city'].unique())
            self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # For both train and test, we have file_name column
        img_name = os.path.join(self.root_dir, self.data.iloc[idx]['filename'])
        image = Image.open(img_name).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        if self.mode == 'train':
            city = self.data.iloc[idx]['city']
            label = self.class_to_idx[city]
            return image, label
        else:
            # Test mode: no label
            return image, self.data.iloc[idx]['filename']


In [5]:
mean = [0.485, 0.456, 0.406]  # Typical ImageNet means
std = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

test_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])


In [19]:
train_dataset = CustomDataset(csv_file=train_csv, root_dir=train_dir, transform=train_transforms, mode='train')
test_dataset = CustomDataset(csv_file=test_csv, root_dir=test_dir, transform=test_transforms, mode='test')


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)


In [7]:
num_classes = len(train_dataset.class_to_idx)  # should be 3 in this case

model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)  # load a pretrained model
model.fc = nn.Linear(model.fc.in_features, num_classes)  # replace the final layer
model = model.to(device)


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [10]:
epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{epochs}"):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc*100:.2f}%")


Training Epoch 1/10: 100%|████████████████████| 219/219 [01:16<00:00,  2.88it/s]


Epoch [1/10], Loss: 0.7456, Accuracy: 70.31%


Training Epoch 2/10: 100%|████████████████████| 219/219 [01:04<00:00,  3.42it/s]


Epoch [2/10], Loss: 0.5755, Accuracy: 77.31%


Training Epoch 3/10: 100%|████████████████████| 219/219 [01:03<00:00,  3.42it/s]


...

Epoch [8/10], Loss: 0.3030, Accuracy: 88.73%


Training Epoch 9/10: 100%|████████████████████| 219/219 [01:06<00:00,  3.28it/s]


Epoch [9/10], Loss: 0.2753, Accuracy: 89.51%


Training Epoch 10/10: 100%|███████████████████| 219/219 [01:08<00:00,  3.18it/s]

Epoch [10/10], Loss: 0.2532, Accuracy: 90.19%


In [11]:
torch.save(model.state_dict(), "model1.pth")


In [12]:
model.load_state_dict(torch.load("model1.pth"))


/var/folders/qx/g4yc694x7cv177p5nntdzmbc0000gn/T/ipykernel_1863/2819550261.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model1.pth")

<All keys matched successfully>

In [20]:
model.eval()
predictions = []

# We have class_to_idx from the train_dataset, let’s invert it for predictions:
idx_to_class = {v: k for k, v in train_dataset.class_to_idx.items()}

with torch.no_grad():
    for images, file_names in tqdm(test_loader, desc="Predicting on Test Set"):
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        
        for f, p in zip(file_names, preds):
            city_name = idx_to_class[p.item()]
            predictions.append((f, city_name))


Predicting on Test Set: 100%|███████████████████| 63/63 [00:10<00:00,  5.90it/s]


In [22]:
pred_df = pd.DataFrame(predictions, columns=["file_name", "city"])
pred_df.to_csv("testpredictions.csv", index=False)
print("Predictions saved to test_predictions.csv")


Predictions saved to test_predictions.csv
